In [1]:
# %%
import numpy as np
import tensorflow as tf
from tensorflow.keras import layers, models
from sklearn.model_selection import train_test_split
from spektral.data import Dataset, Graph
from tqdm import tqdm
from numba import njit
import matplotlib.pyplot as plt

2024-08-21 21:12:16.311168: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-08-21 21:12:16.320010: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-08-21 21:12:16.328749: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-08-21 21:12:16.331535: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1452] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-08-21 21:12:16.339291: I tensorflow/core/platform/cpu_feature_guar

In [2]:
# %%
import os


In [3]:
# %%
os.chdir("/home/philippe/Master/github/SiFiCC-NN/datasets/OptimisedGeometry_CodedMaskHIT_Spot1_2e10_protons_simv5")


In [4]:
# %%
# Load data from files
def load_data():
    adjacency_list = np.load("A.npy")  
    node_attributes = np.load("node_attributes.npy")  
    node_indicator = np.load("node_indicator.npy")
    edge_attributes = np.load("edge_attributes.npy")  
    edge_indicator = np.load("edge_indicator.npy")
    return adjacency_list, node_attributes, node_indicator, edge_attributes, edge_indicator


In [5]:

@njit
def create_adjacency_matrix(edges, num_nodes):
    adj_matrix = np.zeros((num_nodes, num_nodes), dtype=np.float32)
    for i in range(edges.shape[0]):
        src, dst = edges[i]
        adj_matrix[src, dst] = 1
    return adj_matrix


def find_graph_start_indices(indicator_array):
    # Find indices where the graph index changes (i.e., where a new graph begins)
    graph_start_indices = np.where(np.diff(indicator_array, prepend=-1) != 0)[0]
    return graph_start_indices

class MyGraphDataset(Dataset):
    def __init__(self, adjacency_list, node_attributes, node_indicator, edge_attributes, edge_indicator, **kwargs):
        self.adjacency_list = adjacency_list
        self.node_attributes = node_attributes
        self.node_indicator = node_indicator
        self.edge_attributes = edge_attributes
        self.edge_indicator = edge_indicator
        

        super().__init__(**kwargs)

    def read(self):
        graphs = []
        # Compute start indices for nodes and edges
        self.node_start_indices = find_graph_start_indices(self.node_indicator)
        self.edge_start_indices = find_graph_start_indices(self.edge_indicator)
        
        self.num_graphs = len(self.node_start_indices)  # Number of graphs

        # Iterate over each graph index
        for i in tqdm(range(self.num_graphs), desc="Populating Graphs"):
            # Determine the start and end indices for the current graph
            node_start = self.node_start_indices[i]
            node_end = self.node_start_indices[i + 1] if i + 1 < self.num_graphs else len(self.node_indicator)
            
            edge_start = self.edge_start_indices[i]
            edge_end = self.edge_start_indices[i + 1] if i + 1 < self.num_graphs else len(self.edge_indicator)

            # Extract node and edge attributes for the current graph
            x = self.node_attributes[node_start:node_end]
            e = self.edge_attributes[edge_start:edge_end]

            # Extract edges for the current graph
            edges = self.adjacency_list[edge_start:edge_end]

            # Determine the number of nodes in the current graph
            num_nodes = node_end - node_start

            # Create adjacency matrix using Numba-optimized function
            adj_matrix = create_adjacency_matrix(edges, num_nodes)

            # Create Graph object and append to the list
            graph = Graph(x=x, a=adj_matrix, e=e)
            graphs.append(graph)

        return graphs

In [6]:

# Load data
adjacency_list, node_attributes, node_indicator, edge_attributes, edge_indicator = load_data()


In [7]:
# Create dataset
dataset = MyGraphDataset(adjacency_list, node_attributes, node_indicator, edge_attributes, edge_indicator)


Populating Graphs: 100%|██████████| 993783/993783 [00:03<00:00, 293883.55it/s]


In [8]:
import tensorflow as tf
from tensorflow.keras import layers, models

class GATLayer(layers.Layer):
    def __init__(self, num_heads, num_out_features, **kwargs):
        super(GATLayer, self).__init__(**kwargs)
        self.num_heads = num_heads
        self.num_out_features = num_out_features

    def build(self, input_shape):
        num_nodes = input_shape[0][1]
        num_in_features = input_shape[0][-1]
        self.attn_weights = self.add_weight(shape=(num_in_features, self.num_out_features),
                                            initializer='glorot_uniform', name='attn_weights')
        self.attn_bias = self.add_weight(shape=(self.num_out_features,), initializer='zeros', name='attn_bias')
        super(GATLayer, self).build(input_shape)

    def call(self, inputs):
        features, adj = inputs
        features_transformed = tf.matmul(features, self.attn_weights)
        attn_scores = tf.matmul(features_transformed, features_transformed, transpose_b=True)
        attn_scores += self.attn_bias
        attn_scores = tf.nn.softmax(attn_scores, axis=-1)
        aggregated_features = tf.matmul(attn_scores, features_transformed)
        return aggregated_features

In [9]:
def build_gat_model(num_node_features, num_edge_features, num_out_features):
    features_input = layers.Input(shape=(None, num_node_features), name='node_features')
    adj_input = layers.Input(shape=(None, None), name='adj_matrix')

    x = GATLayer(num_heads=1, num_out_features=224)([features_input, adj_input])
    x = layers.GlobalAveragePooling1D()(x)
    x = layers.Dense(64, activation='relu')(x)
    x = layers.Dense(num_out_features, activation='relu')(x)
    edge_features_output = layers.Dense(num_edge_features, activation='linear')(x)

    model = models.Model(inputs=[features_input, adj_input], outputs=edge_features_output)
    model.compile(optimizer='adam', loss='mse')
    return model


In [10]:


# Preparing the data for training
def prepare_data(graphs):
    node_features = []
    adj_matrices = []
    edge_features = []

    for graph in tqdm(graphs):
        node_features.append(graph.x)
        adj_matrices.append(graph.a)
        edge_features.append(graph.e)

    return np.array(node_features, dtype=object), np.array(adj_matrices, dtype=object), np.array(edge_features, dtype=object)


In [12]:
# Extract node features, adjacency matrices, and edge features
node_features, adj_matrices, edge_features = prepare_data(dataset)

100%|██████████| 993783/993783 [00:00<00:00, 1806794.53it/s]


In [13]:
# Train-test split
X_train, X_test, A_train, A_test, y_train, y_test = train_test_split(
    node_features, adj_matrices, edge_features, test_size=0.2, random_state=42
)


In [14]:
# Building the GAT model
num_node_features = node_features[0].shape[1]
num_edge_features = edge_features[0].shape[1]
num_out_features = num_edge_features


In [15]:
model = build_gat_model(num_node_features, num_edge_features, num_out_features)


I0000 00:00:1724267592.599095    2669 cuda_executor.cc:1015] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
I0000 00:00:1724267592.631765    2669 cuda_executor.cc:1015] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
I0000 00:00:1724267592.637290    2669 cuda_executor.cc:1015] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
I0000 00:00:1724267592.641780    2669 cuda_executor.cc:1015] successful NUMA node read from SysFS ha

In [16]:
# Training the model
history = model.fit([X_train, A_train], y_train, epochs=50, batch_size=32, validation_split=0.1, verbose=1)


ValueError: Failed to convert a NumPy array to a Tensor (Unsupported object type numpy.ndarray).